# Credit Risk: Resampling Techniques Notebook
 
This notebook seeks to utilize the Imbalanced Learn library to resample the given LendingClub data. Additionally, the Imbalanced Learn library will be used to build and evaluate logistic regression classifiers through the resampled data. 

---

## Import the Required Libraries & Dependencies


In [7]:
# Import Warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Import Libraries and Dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [9]:
# Import SK-LEARN & IM-LEARN libraries 
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


---

## Read the CSV and Perform Basic Data Cleaning

In [10]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

In [11]:
# Display the df
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


### Brief Data Cleaning

**Important Section Note:** 

It has been pre-determined the given LendingClub data has previously undergone cleaning procedures. The following section iterates the cleaning process to present possible data cleaning methods if it was required. 

#### Data Cleaning Part 1: Remove Null Values 

In [12]:
# Identify null values in the data set
# df = df.isnull.sum()

# Remove null values from the data set
# df = df.dropna(inplace = True)

# Verfiy null values have been removed from the data set 
# df.isnull().sum()

#### Data Cleaning Part 2: Identify Column Errors

The following data cleaning section cotnains a hypothetical scenario that could be present in a dataset. 

**Hypothetical Scenario:**

The section column: "borrower_income", returns an object and contains '$'

In [13]:
# Identify column errors 
 
# Check the data types
# df.dtypes

# Remove the '$' in the "borrower_income" column 
# df['borrower_income'] = df['borrower_income'].str.replace('$', '')

# Display the df
# df.head()

# The column 'borrower_income' returns an object data type, change it to a float 
# df['borrower_income'] = df['borrower_income'].astype('float')

# Check if the data types are correct 
# df.dtypes


#### Data Cleaning Part 3: Summary Statistics

In [14]:
# Summary statistics

# df[df['loan_status'] == "low_risk"].describe()
# df[df['loan_status'] == "high_risk"].describe()


---

## Simple Logistic Regression

### Split the Data into Training and Testing

Split the data into training and testing categories in order to create the Simple Logistic Regression model. 

In [15]:
# Create our features
X = df.copy()
X = X.drop(columns = 'loan_status')
X = pd.get_dummies(X)

# Create our target
y = df['loan_status']

In [16]:
# Display Summary Statistics 
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [17]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

### Split X_train, X_test, y_train, and y_test

In [18]:
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split

In [19]:
# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Shape X_train
X_train.shape

(58152, 10)

In [20]:
# View the count for y_train
Counter(y_train)

Counter({'low_risk': 56277, 'high_risk': 1875})



## Simple Logistic Regression

Using the SKLearn library, create a simple logisitc regression model to classify data points and make predicitons for the LendingClub data. 

In [21]:
# Import the logistic regression model (LR)
from sklearn.linear_model import LogisticRegression

In [22]:
# Set the model variable LR (logistic regression)
LR = LogisticRegression(solver='lbfgs', random_state=1, max_iter=100)

# Fit the model
LR.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Balanced Accuracy Score

In [23]:
# Set a predict model
y_pred = LR.predict(X_test)

# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

### Confusion Matrix

In [24]:
# Display the confusion matrix (CM)
CM = confusion_matrix(y_test, y_pred)

# Create a confusion matrix df (CM_df)
CM_df = pd.DataFrame(CM, 
                     index = ["High Risk (Predicted Positive)", "Low RIsk (Predicted Negative)"], 
                     columns = ["Positive", "Negative"])
# Display the CM_df
display(CM_df)


,Positive,Negative
High Risk (Predicted Positive),571,54
Low RIsk (Predicted Negative),93,18666


### Classification Report

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384




---

## Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests


## Naive Random Oversampling

In [26]:
# Import the Random Over Sampler (ROS)
from imblearn.over_sampling import RandomOverSampler

In [27]:
# Resample the training data with the Random Over Sampler (ROS)
ROS = RandomOverSampler(random_state=1)

# Fit the ROS model
X_resampled, y_resampled = ROS.fit_resample(X_train, y_train)

# y_resampled counter 
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [28]:
# Train the Logistic Regression model (LR) using the resampled data
LR = LogisticRegression(solver='lbfgs', random_state=1, max_iter=100)

# Fit the model
LR.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

### Balanced Accuracy Score

In [29]:
# Calculated the balanced accuracy score
y_pred = LR.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9947746894823818

### Confusion Matrix

In [30]:
# Display the confusion matrix (CM)
CM_ROS = confusion_matrix(y_test, y_pred)

# Create a confusion matrix df (CM_df)
CM_ROS_df = pd.DataFrame(CM_ROS, 
                         index = ["High Risk (Predicted Positive)", "Low RIsk (Predicted Negative)"], 
                         columns = ["Positive", "Negative"])
# Display the CM_df
display(CM_ROS_df)

,Positive,Negative
High Risk (Predicted Positive),622,3
Low RIsk (Predicted Negative),106,18653


### Classification Report

In [31]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



## SMOTE Oversampling

In [32]:
# SMOTE Import
from imblearn.over_sampling import SMOTE

In [33]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train)

# Use counter to count y_resampled
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [34]:
# Train the Logistic Regression model (LR) using the resampled data
LR = LogisticRegression(solver='lbfgs', random_state=1, max_iter=100)

# Fit the model
LR.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

### Balanced Accuracy Score

In [35]:
# Calculated the balanced accuracy score
y_pred = LR.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9948279972279972

### Confusion Matrix

In [36]:
# Display the confusion matrix (CM)
CM_SMOTE = confusion_matrix(y_test, y_pred)

# Create a confusion matrix df (CM_df)
CM_SMOTE_df = pd.DataFrame(CM_SMOTE, 
                           index = ["High Risk (Predicted Positive)", "Low RIsk (Predicted Negative)"], 
                           columns = ["Positive", "Negative"])
# Display the CM_df
display(CM_SMOTE_df)

,Positive,Negative
High Risk (Predicted Positive),622,3
Low RIsk (Predicted Negative),104,18655


### Clasification Report

In [37]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384




---

## Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests


In [38]:
# Import Cluster Centroids (CC)
from imblearn.under_sampling import ClusterCentroids

In [39]:
# Resample the data using the ClusterCentroids resampler (CC)
CC = ClusterCentroids(random_state=1)

# Fit the model
X_resampled, y_resampled = CC.fit_resample(X_train, y_train)

# count y_resampled
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [40]:
# Train the Logistic Regression model (LR) using the resampled data
LR = LogisticRegression(solver='lbfgs', random_state=1, max_iter=100)

# Fit the model
LR.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

### Balanced Accuracy Score

In [41]:
# Calculated the balanced accuracy score
y_pred = LR.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9828813049736127

### Confusion Matrix

In [42]:
# Display the confusion matrix (CM)
CM_CC = confusion_matrix(y_test, y_pred)

# Create a confusion matrix df (CM_df)
CM_CC_df = pd.DataFrame(CM_CC, 
                        index = ["High Risk (Predicted Positive)", "Low RIsk (Predicted Negative)"], 
                        columns = ["Positive", "Negative"])
# Display the CM_df
display(CM_CC_df)

,Positive,Negative
High Risk (Predicted Positive),607,18
Low RIsk (Predicted Negative),102,18657


### Classification Report

In [43]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.96       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384




---

## Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests


In [44]:
# Import SMOTEENN (SM)
from imblearn.combine import SMOTEENN

In [45]:
# Resample the training data with SMOTEENN (SM)
SM = SMOTEENN(random_state=1)

# Fit the model
X_resampled, y_resampled = SM.fit_resample(X_train, y_train)

# Count y_resampled
Counter(y_resampled)

Counter({'high_risk': 55538, 'low_risk': 55928})

In [46]:
# Train the Logistic Regression model (LR) using the resampled data
LR = LogisticRegression(solver='lbfgs', random_state=1, max_iter=100)

# Fit the model
LR.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

### Balanced Accuracy Score

In [47]:
# Calculated the balanced accuracy score
y_pred = LR.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.994748035609574

### Confusion Matrix

In [48]:
# Display the confusion matrix (CM)
CM_SMOTEENN = confusion_matrix(y_test, y_pred)

# Create a confusion matrix df (CM_df)
CM_SMOTEENN_df = pd.DataFrame(CM_SMOTEENN, 
                              index = ["High Risk (Predicted Positive)", "Low RIsk (Predicted Negative)"], 
                              columns = ["Positive", "Negative"])
# Display the CM_df
display(CM_SMOTEENN_df)

,Positive,Negative
High Risk (Predicted Positive),622,3
Low RIsk (Predicted Negative),107,18652


### Classification Report

In [49]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384




---

## Conclusion

1. Which model had the best balanced accuracy score?
2. Which model had the best recall score?
3. Which model had the best geometric mean score?


### Balanced Accuracy Score Values:

| Classifier | Balanced Accuracy Score |
| --- | --- | 
| Simple Logistic Regression | 0.9543211898288821 | 
| Naive Random Oversampling | 0.9543211898288821 | 
| SMOTE | 0.9948279972279972 |
| Cluster Centroids | 0.9948279972279972 | 
| SMOTEENN | 0.9828813049736127 | 


The models with the **best balanced accuracy score** are SMOTE oversampling (99.48%) and Cluster Centroids undersampling (99.48%). 

### Recall Score Values: 
    
| Classifier | High Risk | Low Risk |
| --- | --- | --- |
| Simple Logistic Regression | 0.91 | 0.99 |
| Naive Random Oversampling | 0.99 | 0.99 |
| SMOTE | 0.99 | 0.99 |
| Cluster Centroids | 0.97 | 0.99 |
| SMOTEENN | 1.00 | 0.99 |


The model with the **best recall score** is SMOTEENN. However, both oversampling models; Naive Random Oversampling and SMOTE have high recall scores as well. 

### Geometric Mean Score Values: 

| Classifier | High Risk | Low Risk |
| --- | --- | --- |
| Simple Logistic Regression | 0.95 | 0.95 |
| Naive Random Oversampling | 1.00 | 0.99 |
| SMOTE | 1.00 | 0.99 |
| Cluster Centroids | 0.98 | 0.98 |
| SMOTEENN | 0.99 | 0.99 |


The **best geometric mean score** are the oversmapling models; Naive Random Oversampling and SMOTE. 

---